In [4]:
# Projeto Flashcard Japonês para Iniciantes - Aprimorado

# Este notebook implementa um sistema aprimorado de flashcards para aprender Hiragana, Katakana e vocabulário básico de japonês em tópicos específicos, utilizando o Google AI SDK (ADK) para gerar opções de múltipla escolha e mensagens motivacionais com iconografia.

# **Desenvolvido por:** Agente Agente_dev (Base) & Agente Agente_aprimorador (Aprimoramento)

# @title Configuração Inicial e Instalação
!pip install google-generativeai -q

import os
import random
import google.generativeai as genai
from google.colab import userdata
from IPython.display import display, Markdown

# @title Configuração da API Key e Inicialização do Modelo
GOOGLE_API_KEY = None
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key carregada com sucesso.")
    model = genai.GenerativeModel('gemini-1.5-flash')
    print("Modelo generativo inicializado.")
except userdata.SecretNotFoundError:
    print("ATENÇÃO: A API Key do Google não foi encontrada nas secrets do Colab.")
    print("Por favor, adicione sua chave API como uma secret chamada GOOGLE_API_KEY.")
    print("Saiba mais aqui: https://colab.research.google.com/notebooks/snippets/secrets.ipynb")
    model = None
    print("Modelo generativo não inicializado. Funcionalidades que dependem do modelo podem não funcionar.")
except Exception as e:
    print(f"ATENÇÃO: Erro ao inicializar o modelo generativo: {e}")
    model = None
    print("Modelo generativo não inicializado devido a erro.")


API Key carregada com sucesso.
Modelo generativo inicializado.


In [5]:
# @title Dados para os Flashcards
# Expandindo os dados de Hiragana e Katakana (incluindo alguns dakuon/handakuon básicos)
hiragana_chart = {
    'あ': 'a', 'い': 'i', 'う': 'u', 'え': 'e', 'お': 'o',
    'か': 'ka', 'き': 'ki', 'く': 'ku', 'け': 'ke', 'こ': 'ko',
    'さ': 'sa', 'し': 'shi', 'す': 'su', 'せ': 'se', 'そ': 'so',
    'た': 'ta', 'ち': 'chi', 'つ': 'tsu', 'て': 'te', 'と': 'to',
    'な': 'na', 'に': 'ni', 'ぬ': 'nu', 'ね': 'ne', 'の': 'no',
    'は': 'ha', 'ひ': 'hi', 'ふ': 'fu', 'へ': 'he', 'ほ': 'ho',
    'ま': 'ma', 'み': 'mi', 'む': 'mu', 'め': 'me', 'も': 'mo',
    'や': 'ya', 'ゆ': 'yu', 'よ': 'yo',
    'ら': 'ra', 'り': 'ri', 'る': 'ru', 'れ': 're', 'ろ': 'ro',
    'わ': 'wa', 'を': 'wo', 'ん': 'n',
    'が': 'ga', 'ぎ': 'gi', 'ぐ': 'gu', 'げ': 'ge', 'ご': 'go',
    'ざ': 'za', 'じ': 'ji', 'ず': 'zu', 'ぜ': 'ze', 'ぞ': 'zo',
    'だ': 'da', 'ぢ': 'ji', 'づ': 'zu', 'で': 'de', 'ど': 'do',
    'ば': 'ba', 'び': 'bi', 'ぶ': 'bu', 'べ': 'be', 'ぼ': 'bo',
    'ぱ': 'pa', 'ぴ': 'pi', 'ぷ': 'pu', 'ぺ': 'pe', 'ぽ': 'po',
    'きゃ': 'kya', 'きゅ': 'kyu', 'きょ': 'kyo',
    'しゃ': 'sha', 'しゅ': 'shu', 'しょ': 'sho',
    'ちゃ': 'cha', 'ちゅ': 'chu', 'ちょ': 'cho',
    'にゃ': 'nya', 'にゅ': 'nyu', 'にょ': 'nyo',
    'ひゃ': 'hya', 'ひゅ': 'hyu', 'ひょ': 'hyo',
    'みゃ': 'mya', 'みゅ': 'myu', 'みょ': 'myo',
    'りゃ': 'rya', 'りゅ': 'ryu', 'りょ': 'ryo',
    'ぎゃ': 'gya', 'ぎゅ': 'gyu', 'ぎょ': 'gyo',
    'じゃ': 'ja', 'じゅ': 'ju', 'じょ': 'jo',
    'びゃ': 'bya', 'びゅ': 'byu', 'びょ': 'byo',
    'ぴゃ': 'pya', 'ぴゅ': 'pyu', 'ぴょ': 'pyo',
}

katakana_chart = {
    'ア': 'a', 'イ': 'i', 'ウ': 'u', 'エ': 'e', 'オ': 'o',
    'カ': 'ka', 'キ': 'ki', 'ク': 'ku', 'ケ': 'ke', 'コ': 'ko',
    'サ': 'sa', 'シ': 'shi', 'ス': 'su', 'セ': 'se', 'ソ': 'so',
    'タ': 'ta', 'チ': 'chi', 'ツ': 'tsu', 'テ': 'te', 'ト': 'to',
    'ナ': 'na', 'ニ': 'ni', 'ヌ': 'nu', 'ネ': 'ne', 'ノ': 'no',
    'ハ': 'ha', 'ヒ': 'hi', 'フ': 'fu', 'ヘ': 'he', 'ホ': 'ho',
    'マ': 'ma', 'ミ': 'mi', 'ム': 'mu', 'メ': 'me', 'モ': 'mo',
    'ヤ': 'ya', 'ユ': 'yu', 'ヨ': 'yo',
    'ラ': 'ra', 'リ': 'ri', 'ル': 'ru', 'レ': 're', 'ロ': 'ro',
    'ワ': 'wa', 'ヲ': 'wo', 'ン': 'n',
    'ガ': 'ga', 'ギ': 'gi', 'グ': 'gu', 'ゲ': 'ge', 'ゴ': 'go',
    'ザ': 'za', 'ジ': 'ji', 'ズ': 'zu', 'ゼ': 'ze', 'ゾ': 'zo',
    'ダ': 'da', 'ヂ': 'ji', 'ヅ': 'zu', 'デ': 'de', 'ド': 'do',
    'バ': 'ba', 'ビ': 'bi', 'ブ': 'bu', 'ベ': 'be', 'ボ': 'bo',
    'パ': 'pa', 'ピ': 'pi', 'プ': 'pu', 'ペ': 'pe', 'ポ': 'po',
    'キャ': 'kya', 'キュ': 'kyu', 'キョ': 'kyo',
    'シャ': 'sha', 'シュ': 'shu', 'ショ': 'sho',
    'チャ': 'cha', 'チュ': 'chu', 'チョ': 'cho',
    'ニャ': 'nya', 'ニュ': 'nyu', 'ニョ': 'nyo',
    'ヒャ': 'hya', 'ヒュ': 'hyu', 'ヒョ': 'hyo',
    'ミャ': 'mya', 'ミュ': 'myu', 'ミョ': 'myo',
    'リャ': 'rya', 'リュ': 'ryu', 'リョ': 'ryo',
    'ギャ': 'gya', 'ギュ': 'gyu', 'ギョ': 'gyo',
    'ジャ': 'ja', 'ジュ': 'ju', 'ジョ': 'jo',
    'ビャ': 'bya', 'ビュ': 'byu', 'ビョ': 'byo',
    'ピャ': 'pya', 'ピュ': 'pyu', 'ピョ': 'pyo',
}

# Novos tópicos de vocabulário (com pelo menos 10 itens cada para maior variedade de opções)
vocab_animals = {
    '犬': 'Cachorro', '猫': 'Gato', '鳥': 'Pássaro', '魚': 'Peixe', '兎': 'Coelho',
    '猿': 'Macaco', '象': 'Elefante', '馬': 'Cavalo', '牛': 'Vaca', '豚': 'Porco',
    '虎': 'Tigre', '獅子': 'Leão', '狐': 'Raposa', '狼': 'Lobo', '熊': 'Urso',
}

vocab_verbs = {
    '食べる': 'Comer', '飲む': 'Beber', '話す': 'Falar', '見る': 'Ver/Olhar', '聞く': 'Ouvir/Perguntar',
    '行く': 'Ir', '来る': 'Vir', 'する': 'Fazer', '勉強する': 'Estudar', '読む': 'Ler',
    '書く': 'Escrever', '買う': 'Comprar', '売る': 'Vender', '待つ': 'Esperar', '立つ': 'Levantar',
}

vocab_colors = {
    '赤': 'Vermelho', '青': 'Azul', '緑': 'Verde', '黄色': 'Amarelo', '黒': 'Preto',
    '白': 'Branco', '茶色': 'Marrom', '紫': 'Roxo/Violeta', 'オレンジ': 'Laranja', 'ピンク': 'Rosa',
    '灰色': 'Cinza', '金色': 'Dourado', '銀色': 'Prateado', '透明': 'Transparente', '紺色': 'Azul marinho',
}

# Combinar todos os dados por tópico
all_data_topics = {
    'Hiragana': hiragana_chart,
    'Katakana': katakana_chart,
    'Vocabulário - Animais': vocab_animals,
    'Vocabulário - Verbos': vocab_verbs,
    'Vocabulário - Cores': vocab_colors,
}

# Kaomoji básicos para diferentes sentimentos
kaomoji_happy = [ "(  *̀ᴗ  *́)و", "(^o^)", "ヾ(･ω･*)ﾉ", "(*´▽`*)" ]
kaomoji_motivated = [ "(ง   *̀_  *́)ง", "٩(｡  *̀Д  *́｡)و", "୧(๑  *̀ᗝ  *́)૭" ]
kaomoji_neutral = [ "(-ω-`)", "(・_・;)", "(._.)" ]

In [6]:
# @title Funções do Flashcard
def generate_flashcard_data(topic):
    """Gera dados aleatórios para um flashcard dentro de um tópico específico."""
    data_source = all_data_topics.get(topic)
    if not data_source:
        print(f"Erro: Tópico '{topic}' não encontrado.")
        return None

    # Seleciona um item aleatório do dicionário do tópico
    question, answer = random.choice(list(data_source.items()))

    return {'type': topic, 'question': question, 'answer': answer}

def generate_options(correct_answer, topic, num_options=4):
    """Gera opções de múltipla escolha, incluindo a resposta correta, usando o modelo ou fallback."""
    options = [correct_answer]
    data_source = all_data_topics.get(topic)
    if not data_source:
         print(f"Erro ao gerar opções: Tópico '{topic}' não encontrado.")
         return [correct_answer] # Retorna apenas a resposta correta em caso de erro

    all_possible_answers = list(data_source.values())

    if model is not None and len(all_possible_answers) > num_options: # Só tenta usar o modelo se houver opções suficientes
        try:
            prompt = f"""
            Gere {num_options - 1} opções *incorretas* para um flashcard de japonês sobre "{topic}".
            A resposta *correta* é "{correct_answer}".
            As opções devem ser do mesmo tipo do flashcard (romanji para Hiragana/Katakana, tradução para vocabulário).
            As opções devem ser distintas da resposta correta e entre si.
            Responda APENAS com uma lista Python de strings. Exemplo: ["opção1", "opção2", "opção3"].
            """
            response = model.generate_content(prompt)
            incorrect_options_text = response.text.strip()

            try:
                # Usar eval() com cautela, idealmente usar um parser JSON seguro se o modelo garantir JSON
                # Para este caso específico e ambiente controlado (Colab), eval pode ser aceitável.
                # Se a resposta do modelo for garantidamente JSON, usar json.loads() seria mais seguro.
                # Verifica se a resposta parece uma lista antes de eval
                if incorrect_options_text.startswith('[') and incorrect_options_text.endswith(']'):
                    incorrect_options = eval(incorrect_options_text)
                    if isinstance(incorrect_options, list):
                         # Filtra opções geradas que possam ser a resposta correta ou já na lista
                        incorrect_options = [opt for opt in incorrect_options if opt != correct_answer and opt not in options]
                        options.extend(incorrect_options[:num_options - 1])
                    else:
                        print(f"Aviso: Resposta do modelo não é uma lista: {incorrect_options_text}. Usando fallback.")
                else:
                     print(f"Aviso: Formato de resposta do modelo inesperado: {incorrect_options_text}. Usando fallback.")

            except Exception as e:
                 print(f"Aviso: Erro ao processar a resposta do modelo: {e}. Resposta bruta: {incorrect_options_text}. Usando fallback.")

        except Exception as e:
            print(f"Aviso: Erro ao chamar o modelo de IA para gerar opções: {e}. Usando fallback.")

    # Fallback: Adiciona opções aleatórias do mesmo tópico se necessário
    while len(options) < num_options:
        # Pega uma resposta aleatória que não seja a correta e não esteja já nas opções
        fallback_option = random.choice(all_possible_answers)
        if fallback_option != correct_answer and fallback_option not in options:
            options.append(fallback_option)

    random.shuffle(options)
    return options


def display_flashcard(card_data, options):
    """Exibe o flashcard usando Markdown com o tópico e opções."""
    print("\n--- Novo Flashcard ---")

    question_style = "font-size: 6em;" if card_data['type'] in ['Hiragana', 'Katakana'] else "font-size: 4em;"

    markdown_output = f"""
## Tópico: {card_data['type']}
## Qual a {'romanização de' if card_data['type'] in ['Hiragana', 'Katakana'] else 'significado de'}:

# <span style="{question_style}">{card_data['question']}</span>

**Opções:**
"""
    for i, option in enumerate(options):
        markdown_output += f"- **{i + 1}**: {option}\n"

    display(Markdown(markdown_output))


def get_user_answer(options):
    """Obtém a resposta numérica do usuário e retorna a opção correspondente."""
    while True:
        try:
            choice = int(input(f"Digite o número da sua resposta (1-{len(options)}): "))
            if 1 <= choice <= len(options):
                return options[choice - 1]
            else:
                print("Escolha inválida. Por favor, digite um número dentro do intervalo.")
        except ValueError:
            print("Entrada inválida. Por favor, digite um número.")

def generate_motivational_message(score_percentage):
    """Gera uma mensagem motivacional usando o modelo de IA, incluindo Kaomoji."""
    chosen_kaomoji = random.choice(kaomoji_motivated) # Kaomoji padrão

    if model is not None:
        try:
            prompt = f"""
            Crie uma mensagem motivacional curta e encorajadora para alguém aprendendo japonês.
            O usuário acabou de completar um ciclo de estudo com {score_percentage:.0f}% de acertos.
            A mensagem deve ser em Português e também em Japonês (com romanji e tradução).
            Adapte a mensagem um pouco dependendo se a pontuação foi alta (mais de 70%), média (40-70%), ou baixa (menos de 40%).
            Inclua um ou dois Kaomoji (emojis japoneses) relevantes no final da mensagem em japonês.
            Formato da resposta:
            Português: [Mensagem em Português]
            Japonês: [Mensagem em Japonês] ([Romanji]) - [Tradução] [Kaomoji(s)]
            """
            response = model.generate_content(prompt)
            message_text = response.text.strip()

            # Tenta parsear a resposta
            portuguese_msg = "Mensagem motivacional em Português."
            japanese_msg_full = "頑張って！ (Gambatte!) - Continue assim!"

            lines = message_text.split('\n')
            for line in lines:
                if line.startswith("Português:"):
                    portuguese_msg = line.replace("Português:", "").strip()
                elif line.startswith("Japonês:"):
                    japanese_msg_full = line.replace("Japonês:", "").strip()
                    # Lógica simplificada: tenta encontrar um Kaomoji conhecido no final
                    found_kaomoji = None
                    for k in kaomoji_happy + kaomoji_motivated + kaomoji_neutral:
                        if japanese_msg_full.endswith(k):
                            chosen_kaomoji = k
                            japanese_msg_full = japanese_msg_full[:-len(k)].strip()
                            found_kaomoji = True
                            break
                    if not found_kaomoji: # Se o modelo gerou um Kaomoji não reconhecido ou nenhum, escolhemos um
                         if score_percentage > 70:
                             chosen_kaomoji = random.choice(kaomoji_happy)
                         elif score_percentage > 40:
                             chosen_kaomoji = random.choice(kaomoji_motivated)
                         else:
                             chosen_kaomoji = random.choice(kaomoji_neutral)


        except Exception as e:
            print(f"Aviso: Erro ao gerar mensagem motivacional com IA: {e}. Usando fallback.")
             # Fallback de mensagem se a IA falhar
            portuguese_msg = "Ótimo esforço!"
            japanese_msg_full = "次は頑張って！ (Tsugi wa gambatte!) - Dê o seu melhor da próxima vez!"
             # Seleciona Kaomoji baseado na pontuação no fallback
            if score_percentage > 70:
                chosen_kaomoji = random.choice(kaomoji_happy)
            elif score_percentage > 40:
                chosen_kaomoji = random.choice(kaomoji_motivated)
            else:
                chosen_kaomoji = random.choice(kaomoji_neutral)


    else:
        # Fallback completo se o modelo não foi inicializado
        portuguese_msg = "Bom trabalho!"
        japanese_msg_full = "頑張って！ (Gambatte!) - Continue assim!"
        # Seleciona Kaomoji baseado na pontuação no fallback
        if score_percentage > 70:
            chosen_kaomoji = random.choice(kaomoji_happy)
        elif score_percentage > 40:
            chosen_kaomoji = random.choice(kaomoji_motivated)
        else:
            chosen_kaomoji = random.choice(kaomoji_neutral)

    return portuguese_msg, f"{japanese_msg_full} {chosen_kaomoji}"


def select_study_options():
    """Permite ao usuário selecionar o tópico de estudo e a quantidade de flashcards."""
    print("Olá! Bem-vindo ao seu estudo de japonês.")
    print("Por favor, escolha um tópico para estudar:")

    topics = list(all_data_topics.keys())
    for i, topic in enumerate(topics):
        print(f"{i + 1}: {topic} ({len(all_data_topics[topic])} termos)") # Mostra a quantidade de termos

    chosen_topic = None
    while chosen_topic is None:
        try:
            choice = int(input(f"Digite o número do tópico desejado (1-{len(topics)}): "))
            if 1 <= choice <= len(topics):
                chosen_topic = topics[choice - 1]
            else:
                print("Escolha inválida. Por favor, digite um número dentro do intervalo.")
        except ValueError:
            print("Entrada inválida. Por favor, digite um número.")

    print(f"\nVocê escolheu o tópico: {chosen_topic}")

    # Nível de dificuldade simplificado: apenas quantidade de flashcards
    max_flashcards = len(all_data_topics[chosen_topic])
    num_flashcards = 0
    while num_flashcards <= 0 or num_flashcards > max_flashcards:
        try:
            num_flashcards = int(input(f"Quantos flashcards você gostaria de revisar neste ciclo (máx. {max_flashcards})? "))
            if num_flashcards <= 0:
                print("Por favor, digite um número positivo de flashcards.")
            elif num_flashcards > max_flashcards:
                print(f"Você solicitou mais flashcards do que termos disponíveis neste tópico. Máximo é {max_flashcards}.")
        except ValueError:
            print("Entrada inválida. Por favor, digite um número.")

    return chosen_topic, num_flashcards

In [7]:
# @title Rodar um Ciclo de Estudo
def run_study_cycle():
    """Roda um ciclo de estudo de flashcards com seleção de tópico e quantidade."""
    if model is None:
        print("\nNão é possível iniciar o ciclo de estudo. O modelo generativo não foi inicializado.")
        return

    chosen_topic, num_flashcards = select_study_options()

    correct_answers_count = 0
    total_flashcards = num_flashcards

    print(f"\nIniciando um ciclo de estudo de '{chosen_topic}' com {total_flashcards} flashcards...")

    # Para evitar repetição excessiva no ciclo curto, podemos embaralhar os itens do tópico e pegar os primeiros N
    study_items = random.sample(list(all_data_topics[chosen_topic].items()), num_flashcards)


    for i, (question, answer) in enumerate(study_items):
        card_data = {'type': chosen_topic, 'question': question, 'answer': answer}

        options = generate_options(card_data['answer'], chosen_topic)

        display_flashcard(card_data, options)

        user_answer = get_user_answer(options)

        print(f"\nSua resposta: {user_answer}")
        print(f"Resposta correta: {card_data['answer']}")

        if user_answer == card_data['answer']:
            print("Status: ✅ Correto!")
            correct_answers_count += 1
        else:
            print("Status: ❌ Incorreto.")

        # Pequena pausa para leitura, opcional
        # input("Pressione Enter para continuar...")

    # --- Resumo do Ciclo ---
    print("\n" + "="*30)
    print("   Fim do Ciclo de Estudo   ")
    print("="*30)

    accuracy_percentage = (correct_answers_count / total_flashcards) * 100 if total_flashcards > 0 else 0

    print(f"\nResultados para o tópico '{chosen_topic}':")
    print(f"Total de Flashcards: {total_flashcards}")
    print(f"Acertos: {correct_answers_count}")
    print(f"Erros: {total_flashcards - correct_answers_count}")
    print(f"Porcentagem de Acerto: {accuracy_percentage:.2f}%")

    # Mensagem Motivacional com Kaomoji
    portuguese_motivation, japanese_motivation_with_kaomoji = generate_motivational_message(accuracy_percentage)
    print("\nMensagem Motivacional:")
    print(f"Português: {portuguese_motivation}")
    print(f"Japonês: {japanese_motivation_with_kaomoji}")

    print("\nObrigado por estudar!")


In [8]:
# --- Iniciar o Estudo ---
run_study_cycle()

Olá! Bem-vindo ao seu estudo de japonês.
Por favor, escolha um tópico para estudar:
1: Hiragana (104 termos)
2: Katakana (104 termos)
3: Vocabulário - Animais (15 termos)
4: Vocabulário - Verbos (15 termos)
5: Vocabulário - Cores (15 termos)
Digite o número do tópico desejado (1-5): 3

Você escolheu o tópico: Vocabulário - Animais
Quantos flashcards você gostaria de revisar neste ciclo (máx. 15)? 3

Iniciando um ciclo de estudo de 'Vocabulário - Animais' com 3 flashcards...
Aviso: Formato de resposta do modelo inesperado: ```python
["Cachorro", "Gato", "Zebra"]
```. Usando fallback.

--- Novo Flashcard ---



## Tópico: Vocabulário - Animais
## Qual a significado de:

# <span style="font-size: 4em;">象</span>

**Opções:**
- **1**: Peixe
- **2**: Urso
- **3**: Vaca
- **4**: Elefante


Digite o número da sua resposta (1-4): 3

Sua resposta: Vaca
Resposta correta: Elefante
Status: ❌ Incorreto.
Aviso: Formato de resposta do modelo inesperado: ```python
["Inu", "Ushi", "Tori"]
```. Usando fallback.

--- Novo Flashcard ---



## Tópico: Vocabulário - Animais
## Qual a significado de:

# <span style="font-size: 4em;">猫</span>

**Opções:**
- **1**: Cavalo
- **2**: Urso
- **3**: Gato
- **4**: Cachorro


Digite o número da sua resposta (1-4): 2

Sua resposta: Urso
Resposta correta: Gato
Status: ❌ Incorreto.
Aviso: Formato de resposta do modelo inesperado: ```python
["neko", "tori", "ushi"]
```. Usando fallback.

--- Novo Flashcard ---



## Tópico: Vocabulário - Animais
## Qual a significado de:

# <span style="font-size: 4em;">犬</span>

**Opções:**
- **1**: Porco
- **2**: Coelho
- **3**: Cachorro
- **4**: Tigre


Digite o número da sua resposta (1-4): 3

Sua resposta: Cachorro
Resposta correta: Cachorro
Status: ✅ Correto!

   Fim do Ciclo de Estudo   

Resultados para o tópico 'Vocabulário - Animais':
Total de Flashcards: 3
Acertos: 1
Erros: 2
Porcentagem de Acerto: 33.33%

Mensagem Motivacional:
Português: Mensagem motivacional em Português.
Japonês: 頑張って！ (Gambatte!) - Continue assim! ٩(｡  *̀Д  *́｡)و

Obrigado por estudar!
